# f-GAN with MNIST

* `f-GAN: Training Generative Neural Samplers using Variational Divergence Minimization`, [arXiv:1606.00709](https://arxiv.org/abs/1606.00709)
  * Sebastian Nowozin, Botond Cseke, and Ryota Tomioka
  
* Implemented by [`tf.keras.layers`](https://www.tensorflow.org/api_docs/python/tf/keras/layers) and [`eager execution`](https://www.tensorflow.org/guide/eager).
* Use `transposed_conv2d` and `conv2d` for Generator and Discriminator, respectively.
  * I do not use `dense` layer for model architecture consistency. (So my architecture is different from original dcgan structure)
* based on DCGAN model

## Import modules

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time
import glob

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import PIL
import imageio
from IPython import display

import tensorflow as tf
from tensorflow.keras import layers
tf.enable_eager_execution()

sys.path.append(os.path.dirname(os.path.abspath('.')))
from utils.image_utils import *
from utils.ops import *

os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Setting hyperparameters

In [ ]:
# Training Flags (hyperparameter configuration)
model_name = 'fgan'
train_dir = 'train/' + model_name + '/exp1/'
max_epochs = 50
save_model_epochs = 10
print_steps = 100
save_images_epochs = 1
batch_size = 128
learning_rate_D = 1e-4
learning_rate_G = 1e-4
k = 1 # the number of step of learning D before learning G (Not used in this code)
num_examples_to_generate = 16
noise_dim = 100

## Load the MNIST dataset

In [ ]:
# Load training and eval data from tf.keras
(train_data, train_labels), _ = \
    tf.keras.datasets.mnist.load_data()

train_data = train_data.reshape(-1, MNIST_SIZE, MNIST_SIZE, 1).astype('float32')
train_data = train_data / 255.

## Set up dataset with `tf.data`

### create input pipeline with `tf.data.Dataset`

In [ ]:
#tf.random.set_seed(219)

# for train
N = len(train_data)
train_dataset = tf.data.Dataset.from_tensor_slices(train_data[:N])
train_dataset = train_dataset.shuffle(buffer_size=N)
train_dataset = train_dataset.batch(batch_size=batch_size, drop_remainder=True)
print(train_dataset)

## Create the generator and discriminator models

In [ ]:
class Generator(tf.keras.Model):
  """Build a generator that maps latent space to real space.
    G(z): z -> x
  """
  def __init__(self):
    super(Generator, self).__init__()
    self.conv1 = ConvTranspose(256, 3, padding='valid')
    self.conv2 = ConvTranspose(128, 3, padding='valid')
    self.conv3 = ConvTranspose(64, 4)
    self.conv4 = ConvTranspose(1, 4, apply_batchnorm=False, activation='sigmoid')

  def call(self, inputs, training=True):
    """Run the model."""
    # inputs: [1, 1, 100]
    conv1 = self.conv1(inputs, training=training)           # conv1: [3, 3, 256]
    conv2 = self.conv2(conv1, training=training)            # conv2: [7, 7, 128]
    conv3 = self.conv3(conv2, training=training)            # conv3: [14, 14, 64]
    generated_images = self.conv4(conv3, training=training) # generated_images: [28, 28, 1]
    
    return generated_images

In [ ]:
class Discriminator(tf.keras.Model):
  """Build a discriminator that discriminate real image x whether real or fake.
    D(x): x -> [0, 1]
  """
  def __init__(self):
    super(Discriminator, self).__init__()
    self.conv1 = Conv(64, 4, 2, apply_batchnorm=False, activation='leaky_relu')
    self.conv2 = Conv(128, 4, 2, activation='leaky_relu')
    self.conv3 = Conv(256, 3, 2, padding='valid', activation='leaky_relu')
    self.conv4 = Conv(1, 3, 1, padding='valid', apply_batchnorm=False, activation='none')

  def call(self, inputs, training=True):
    """Run the model."""
    # inputs: [28, 28, 1]
    conv1 = self.conv1(inputs)                            # conv1: [14, 14, 64]
    conv2 = self.conv2(conv1)                             # conv2: [7, 7, 128]
    conv3 = self.conv3(conv2)                             # conv3: [3, 3, 256]
    conv4 = self.conv4(conv3)                             # conv4: [1, 1, 1]
    discriminator_logits = tf.squeeze(conv4, axis=[1, 2]) # discriminator_logits: [1,]
    
    return discriminator_logits

In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
# tf.contrib.eager.defun will be deprecated in TF version 2.0
# Defun for performance boost
generator.call = tf.contrib.eager.defun(generator.call)
discriminator.call = tf.contrib.eager.defun(discriminator.call)

## Define the loss functions and the optimizer

In [ ]:
# tf.keras.losses will be used in TF version 2.0
# use logits for consistency with previous code I made
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def fGANLoss(logits, is_real=True, method='gan'):
  """Computes standard f-GAN.
  See the Table 6. in the f-GAN paper (https://arxiv.org/abs/1606.00709)

  Args:
    logits (`2-rank Tensor`): logits.
    is_real (`bool`): True means `1` labeling, False means `0` labeling.
    method (`string`): ['gan', 'kl', 'rkl', 'js'], this list will be updated

  Returns:
    loss (`0-rank Tensor`): the f-GAN loss value calculated using method you want.
  """
  epsilon = 1e-10
  assert method in ['gan', 'kl', 'rkl', 'js']
  if method == 'gan': # GAN
    activation = lambda x: -tf.math.log(1. + tf.math.exp(-x))
    conjuate = lambda x: -tf.math.log(1. - tf.math.exp(x) + epsilon)
  elif method == 'kl': # Kullback-Leibler (KL)
    activation = lambda x: x
    conjuate = lambda x: tf.math.exp(x - 1.)
  elif method == 'rkl': # Reverse KL
    activation = lambda x: -tf.math.exp(x)
    conjuate = lambda x: −1. − tf.math.log(−x + epsilon)
  elif method == 'js': # Jensen-Shannon
    activation = lambda x: tf.math.log(2.) − tf.math.log(1. + tf.math.exp(−x))
    conjuate = lambda x: −tf.math.log(2. − tf.math.exp(x) + epsilon)
    
    
  if is_real:
    loss = tf.reduce_mean(activation(logits))
  else:
    loss = tf.reduce_mean(-conjuate(activation(logits)))
    
  return loss

In [ ]:
def discriminator_loss(real_logits, fake_logits):
  # losses of real with label "1"
  real_loss = fGANLoss(logits=real_logits, is_real=True)
  # losses of fake with label "0"
  fake_loss = fGANLoss(logits=fake_logits, is_real=False)
  
  return real_loss + fake_loss

In [ ]:
def generator_loss(fake_logits):
  # losses of Generator with label "1" that used to fool the Discriminator
  return fGANLoss(logits=fake_logits, is_real=True)

In [ ]:
#discriminator_optimizer = tf.train.AdamOptimizer(learning_rate_D, beta1=0.5)
discriminator_optimizer = tf.train.RMSPropOptimizer(learning_rate_D)
generator_optimizer = tf.train.AdamOptimizer(learning_rate_G, beta1=0.5)

## Checkpoints (Object-based saving)

In [ ]:
checkpoint_dir = train_dir
if not tf.gfile.Exists(checkpoint_dir):
  tf.gfile.MakeDirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Training

In [ ]:
# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement of the gan.
const_random_vector_for_saving = tf.random.uniform([num_examples_to_generate, 1, 1, noise_dim],
                                                   minval=-1.0, maxval=1.0)

In [ ]:
print('Start Training.')
global_step = tf.train.get_or_create_global_step()
num_batches_per_epoch = int(N / batch_size)
for epoch in range(max_epochs):
  
  for step, images in enumerate(train_dataset):
    start_time = time.time()
    
    # generating noise from a uniform distribution
    noise = tf.random.uniform([batch_size, 1, 1, noise_dim], minval=-1.0, maxval=1.0)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_logits = discriminator(images, training=True)
      fake_logits = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_logits)
      disc_loss = discriminator_loss(real_logits, fake_logits)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.variables)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.variables)

    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.variables))
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.variables),
                                        global_step=global_step)
    
    epochs = epoch + step / float(num_batches_per_epoch)
    duration = time.time() - start_time

    if global_step.numpy() % print_steps == 0:
      display.clear_output(wait=True)
      examples_per_sec = batch_size / float(duration)
      print("Epochs: {:.2f} global_step: {} loss_D: {:.3g} loss_G: {:.3g} ({:.2f} examples/sec; {:.3f} sec/batch)".format(
                epochs, global_step.numpy(), disc_loss, gen_loss, examples_per_sec, duration))
      random_vector_for_sampling = tf.random.uniform([num_examples_to_generate, 1, 1, noise_dim],
                                                     minval=-1.0, maxval=1.0)
      sample_images = generator(random_vector_for_sampling, training=False)
      print_or_save_sample_images(sample_images.numpy(), num_examples_to_generate)

  if (epoch + 1) % save_images_epochs == 0:
    display.clear_output(wait=True)
    print("This images are saved at {} epoch".format(epoch+1))
    sample_images = generator(const_random_vector_for_saving, training=False)
    print_or_save_sample_images(sample_images.numpy(), num_examples_to_generate,
                                is_square=True, is_save=True, epoch=epoch+1,
                                checkpoint_dir=checkpoint_dir)

  # saving (checkpoint) the model every save_epochs
  if (epoch + 1) % save_model_epochs == 0:
    checkpoint.save(file_prefix=checkpoint_prefix)
    
print('Training Done.')

In [ ]:
# generating after the final epoch
display.clear_output(wait=True)
sample_images = generator(const_random_vector_for_saving, training=False)
print_or_save_sample_images(sample_images.numpy(), num_examples_to_generate,
                            is_square=True, is_save=True, epoch=epoch+1,
                            checkpoint_dir=checkpoint_dir)

## Restore the latest checkpoint

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Display an image using the epoch number

In [ ]:
display_image(max_epochs, checkpoint_dir)

## Generate a GIF of all the saved images.

In [ ]:
filename = model_name + '.gif'
generate_gif(filename, checkpoint_dir)

In [ ]:
display.Image(filename=filename + '.png')